In [1]:
import numpy as np
import pandas as pd
from prophet import Prophet
from prophet.plot import plot_plotly
import plotly.offline as py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-v0_8-muted')

df=pd.read_csv("./train.csv").drop(columns=['ID', '소분류', '중분류', '대분류', '브랜드'])
df=df.dropna()

df.set_index('제품', inplace=True)

# 칼럼 이름을 datetime 형식으로 변환
columns = df.columns
df.columns = pd.to_datetime(columns)

# Transpose
df = df.transpose()
df.reset_index(level=0, inplace=True)
df = df.rename(columns={'index': 'ds'})
df.head()

제품,ds,B002-00001-00001,B002-00002-00001,B002-00002-00002,B002-00002-00003,B002-00003-00001,B002-00003-00002,B002-00003-00003,B002-00003-00004,B002-00003-00005,...,B002-03798-00052,B002-03798-00053,B002-03798-00054,B002-03798-00055,B002-03799-00001,B002-03799-00002,B002-03799-00003,B002-03799-00004,B002-03799-00005,B002-03799-00010
0,2022-01-01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2022-01-02,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2022-01-03,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2022-01-04,0,0,0,0,0,0,7,0,15,...,0,0,0,0,0,0,0,0,0,0
4,2022-01-05,0,0,0,0,0,0,21,0,16,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
dsy = []
for i in range(1, len(df.columns)):
    column_name = df.columns[i]
    selected_df = df[['ds', column_name]].copy()  # 'ds'와 해당 열만 선택하여 복사
    selected_df.rename(columns={column_name: 'y'}, inplace=True)  # 열 이름 변경
    dsy.append(selected_df)

# dsy 리스트에는 각 열의 데이터프레임이 저장됨

In [ ]:
print(len(dsy))

# 1. 겹치는 제품명 모두 살려서 구분하기

In [ ]:
df=pd.read_csv("./train.csv").drop(columns=['ID', '소분류', '중분류', '대분류', '브랜드'])
df = df.dropna()
df.set_index('제품', inplace=True)
columns = df.columns
df.columns = pd.to_datetime(columns)

df = df.transpose()

# 겹치는 제품명에 대해서 구분 -> 다른 정보들과 함께 쓰이면 구분 가능해서 방법 찾아봐야함(브랜드 등 정보)
new_columns = [f"{col}_{i}" for i, col in enumerate(df.columns)]
df.columns = new_columns

df.reset_index(level=0, inplace=True)
df = df.rename(columns={'index': 'ds'})

all_forecasts = []  # 모든 예측 결과를 저장할 리스트

# 모든 제품에 대한 작업 반복
for i in range(1, len(df.columns)):
    column_name = df.columns[i]
    selected_df = df[['ds', column_name]].copy()  # 'ds'와 해당 열만 선택하여 복사
    selected_df.rename(columns={column_name: 'y'}, inplace=True)  # 열 이름 변경

    # Prophet 모델 초기화
    my_model = Prophet()

    # 모델 피팅
    my_model.fit(selected_df)

    # 21일 간
    future_dates = my_model.make_future_dataframe(periods=21, freq='D', include_history=False)
    
    # 판매총액 예측
    forecast = my_model.predict(future_dates)

    # 예측 결과를 리스트에 추가
    all_forecasts.append(forecast)

# 모든 예측 결과를 하나의 데이터프레임으로 병합
merged_forecasts = pd.concat(all_forecasts, ignore_index=True)

# 데이터를 CSV 파일로 저장
merged_forecasts.to_csv('merged_forecasts.csv', index=False)

print("모든 예측 결과가 병합되어 'merged_forecasts.csv' 파일로 저장되었습니다.")

# 2. mean 평균내서 하나만 살리기

In [ ]:
df=pd.read_csv("./train.csv").drop(columns=['ID', '소분류', '중분류', '대분류', '브랜드'])
df = df.dropna()
df.set_index('제품', inplace=True)
columns = df.columns
df.columns = pd.to_datetime(columns)

# 겹치는 제품명을 그룹화하여 평균 계산
df_grouped = df.groupby(df.index).mean()

df = df_grouped.transpose()

df.reset_index(level=0, inplace=True)
df = df.rename(columns={'index': 'ds'})

# 모든 제품에 대한 작업 반복
for i in range(1, len(df.columns)):
    column_name = df.columns[i]
    selected_df = df[['ds', column_name]].copy()  # 'ds'와 해당 열만 선택하여 복사
    selected_df.rename(columns={column_name: 'y'}, inplace=True)  # 열 이름 변경

    # Prophet 모델 초기화
    my_model = Prophet()

    # 모델 피팅
    my_model.fit(selected_df)

    # 21일 간
    future_dates = my_model.make_future_dataframe(periods=21, freq='D', include_history=False)
    
    # 판매총액 예측
    forecast = my_model.predict(future_dates)

    # 예측 결과를 리스트에 추가
    all_forecasts.append(forecast)

# 모든 예측 결과를 하나의 데이터프레임으로 병합
merged_forecasts = pd.concat(all_forecasts, ignore_index=True)

# 데이터를 CSV 파일로 저장
merged_forecasts.to_csv('merged_forecasts.csv', index=False)

print("모든 예측 결과가 병합되어 'merged_forecasts.csv' 파일로 저장되었습니다.")

In [ ]:
import logging

# 로그 레벨 조정
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)  # WARNING 레벨 이상의 로그만 표시

In [2]:
result = pd.read_csv('merged_forecasts.csv')
result

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-04-05,1.301929,-0.344161,3.686817,1.301929,1.301929,0.336581,0.336581,0.336581,0.336581,0.336581,0.336581,0.0,0.0,0.0,1.638510
1,2023-04-06,1.301528,-0.611197,3.648853,1.301528,1.301528,0.207877,0.207877,0.207877,0.207877,0.207877,0.207877,0.0,0.0,0.0,1.509406
2,2023-04-07,1.301128,-0.607041,3.449199,1.301128,1.301128,0.220428,0.220428,0.220428,0.220428,0.220428,0.220428,0.0,0.0,0.0,1.521556
3,2023-04-08,1.300727,-1.012173,3.081317,1.300727,1.300798,-0.186524,-0.186524,-0.186524,-0.186524,-0.186524,-0.186524,0.0,0.0,0.0,1.114203
4,2023-04-09,1.300326,-1.122315,3.036185,1.300249,1.300517,-0.266977,-0.266977,-0.266977,-0.266977,-0.266977,-0.266977,0.0,0.0,0.0,1.033350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332635,2023-04-21,-0.001974,-0.045554,0.072749,-0.001987,-0.001963,0.013255,0.013255,0.013255,0.013255,0.013255,0.013255,0.0,0.0,0.0,0.011281
332636,2023-04-22,-0.001987,-0.062285,0.055631,-0.002001,-0.001975,-0.002249,-0.002249,-0.002249,-0.002249,-0.002249,-0.002249,0.0,0.0,0.0,-0.004237
332637,2023-04-23,-0.002000,-0.062886,0.056156,-0.002016,-0.001987,-0.002225,-0.002225,-0.002225,-0.002225,-0.002225,-0.002225,0.0,0.0,0.0,-0.004225
332638,2023-04-24,-0.002013,-0.062442,0.053954,-0.002030,-0.001998,-0.002200,-0.002200,-0.002200,-0.002200,-0.002200,-0.002200,0.0,0.0,0.0,-0.004214
